In [21]:
import astropy.units as units
import astropy.constants as constants
import matplotlib.pyplot as plt
import sympy as sym
import numpy as np 
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import requests
import re
import subprocess
import urllib.request
from sympy.abc import *
from bs4 import BeautifulSoup
import csv
cwd = subprocess.os.getcwd()

In [25]:
def half_life_to_lambda(half_life):
    '''
    This function takes the e-folding times of the entire decay chain.
    Returns the half-life of the nuclide
    '''
    return half_life / np.log(2)
nuclide_df = pd.read_csv(cwd + '\\NuclideData.csv').iloc[:,1:]
half_lives = nuclide_df['Half life (years)'].to_numpy() 
nuclide_df["e Folding Time (seconds)"] = half_life_to_lambda(half_lives * units.year.to(units.s))
nuclide_df

,Beta-decay fraction,Average beta decay energy,Beta-decay energy (keV),Half life (years),Isotope,e Folding Time (seconds)
0,0.0,1533.00,-1864.009,1.058382e-06,80Rb,4.818601e+01
1,0.0,5.69,5480.0,1.584404e-07,185Yb,7.213475e+00
2,0.0,6166.00,-335.0623,5.604381e-02,253Es,2.551562e+06
3,1.0,5.69,3951.0,1.901285e-07,195Re,8.656170e+00
4,0.0,5.67,-6989.4042,3.587725e-05,162Yb,1.633419e+03
...,...,...,...,...,...,...
3187,1.0,5.69,11561.224,6.052425e-09,58V,2.755548e-01
3188,1.0,79.50,702.7199,1.066621e-03,127Te,4.856112e+04
3189,0.0,543.30,-5069.1361,7.170000e+05,26Al,3.264357e+13
3190,0.0,477.00,-9447.8181,6.342688e-04,90Mo,2.887698e+04


In [35]:
element_symbols = ['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 
'F', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 
'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 
'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 
'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 
'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 
'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm',
 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 
 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 
 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 
 'Fm', 'Md', 'No', 'Lr', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt',
  'Ds', 'Rg', 'Cn', 'Nh', 'Fl', 'Mc', 'Lv', 'Ts', 'Og', 'N/A']
len(element_symbols)
#to find the daughter nuclide we only need to increment by 1 in the element symbols.
element_symbols[element_symbols.index('Ni')+1]
#remove numbers from string
s = element_symbols
nuclide_df['Daugher Nucleus'] = [re.sub('\D+', '', n) 
  + s[s.index(re.sub('\d+', '', n))+1] if re.sub('\d+', '', n) 
  in s else 'N/A' for n in nuclide_df['Isotope'] ]
def make_decay_chain(isotope, isotope_list, lambda_list, decay_energy_list, daughter_list):
    '''
    This function takes the isotope, e-folding times, and daughter nuclides
    and returns the decay chain.
    '''
    decay_chain = {}
    decay_chain[isotope] = get_isotope_info(isotope, isotope_list = isotope_list, 
        lists_to_search = (lambda_list, decay_energy_list, daughter_list))
    while True:
        isotope = decay_chain[isotope][2]
        try:
            decay_chain[isotope] = get_isotope_info(isotope, 
                isotope_list = isotope_list,
                lists_to_search = (lambda_list, decay_energy_list, daughter_list))
            if(isotope == decay_chain[isotope][2]):
                return decay_chain
        except:
            return pd.DataFrame(decay_chain, index= ("e-Folding Time (seconds)", 
                                "Average beta-decay energy", 
                                "Daughter")).transpose()
def get_isotope_info(isotope, info = None, isotope_column = None,
    dataset = {}, isotope_list = None, lists_to_search = []):                  
  '''
  isotope_list and list_to_search are optional arguments.
  If list_to_search is not provided, then info must be provided.
  If isotope_list is not provided, then dataset and isotope_column
  must be provided.
  '''
  if isotope_list is None:
    isotope_list = list(dataset[isotope_column])
  row = isotope_list.index(isotope)
  if len(lists_to_search) == 0:
    try:
      lists_to_search = list(dataset[info])
    except:
      print("info to search for not entered")
      return
  try: #only works if there are multiple specified lists to search 
    return [target_list[row] for target_list in lists_to_search]
  except:
    return lists_to_search[row]
def clean_time_series(time_series):
    '''
    This function takes the time series of power densities and returns a list
    of lists of the time series converted into floats.
    '''
    split_time_series = time_series.replace('[', ' ').replace(']', ' ').replace('\n', ' ').split(' ') 
    return np.round(np.array([float(power_density)  
                    for power_density in split_time_series 
                    if len(power_density)>0]), 2)

In [36]:
isotope_list = list(nuclide_df['Isotope'])
lambda_list = list(nuclide_df['e Folding Time (seconds)'])
decay_energy_list = list(nuclide_df['Average beta decay energy'])
daughter_list = list(nuclide_df['Daugher Nucleus'])
all_decay_chains = [make_decay_chain(isotope, isotope_list, lambda_list, 
                    decay_energy_list, daughter_list) 
                    for isotope in isotope_list]

In [70]:
file_path = "C:\\Users\\engin\\Documents\\GitHub\\Energy\\power_density_time_series.csv"
with open(file_path, newline='\n') as f:
    reader = csv.reader(f)
    time_series_unclean = list(reader)[0]
power_densities_dict = {}
power_densities_dict['Time (years)'] = np.logspace(0, 9.5, 10**3) * units.second.to(units.year)
for row, chain in enumerate(all_decay_chains):
    power_densities_dict[chain.index[0]] = clean_time_series(time_series_unclean[row])
power_densities_df = pd.DataFrame.from_dict(power_densities_dict).set_index('Time (years)')
transposed_power_densities_df = power_densities_df.transpose()
transposed_power_densities_df

Time (years),3.168809e-08,3.238960e-08,3.310664e-08,3.383955e-08,3.458869e-08,3.535441e-08,3.613708e-08,3.693708e-08,3.775479e-08,3.859061e-08,...,8.228306e+01,8.410464e+01,8.596654e+01,8.786967e+01,8.981492e+01,9.180324e+01,9.383557e+01,9.591290e+01,9.803621e+01,1.002065e+02
80Rb,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
185Yb,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
253Es,435636.52,439796.86,443847.06,447782.55,451599.08,455292.78,458860.17,462298.10,465603.83,468774.97,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
195Re,1550902.27,1551523.68,1551949.04,1552191.70,1552264.34,1552178.95,1551946.84,1551578.60,1551084.14,1550472.69,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162Yb,0.01,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58V,51446960.90,50584906.30,49718680.40,48848591.80,47974961.30,47098120.90,46218414.20,45336196.10,44451832.80,43565701.30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
127Te,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26Al,6433.35,6433.22,6433.10,6432.97,6432.84,6432.71,6432.57,6432.43,6432.29,6432.15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90Mo,10506.59,10506.58,10506.58,10506.58,10506.57,10506.57,10506.56,10506.56,10506.56,10506.55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
power_densities_df

,80Rb,185Yb,253Es,195Re,162Yb,236Bk,238Pa,110Rh,85Zr,151Ba,...,225Ac,158Pr,117Sb,123I,8Li,58V,127Te,26Al,90Mo,200Po
Time (years),,,,,,,,,,,,,,,,,,,,,
3.168809e-08,0.0,0.0,435636.52,1550902.27,0.01,41450532.8,1.66,24304.35,141396.70,673019.51,...,1884568.96,1084.06,322.94,228070688.0,0.0,51446960.9,0.0,6433.35,10506.59,1394383.48
3.238960e-08,0.0,0.0,439796.86,1551523.68,0.01,40422679.9,1.66,20644.00,141394.72,669205.40,...,1870290.94,1084.08,240.41,227516629.0,0.0,50584906.3,0.0,6433.22,10506.58,1235096.61
3.310664e-08,0.0,0.0,443847.06,1551949.04,0.01,39392628.8,1.66,17471.62,141392.70,665329.19,...,1855808.44,1084.10,177.81,226951645.0,0.0,49718680.4,0.0,6433.10,10506.58,1091071.91
3.383955e-08,0.0,0.0,447782.55,1552191.70,0.00,38361178.8,1.66,14732.19,141390.63,661390.38,...,1841121.00,1084.11,130.64,226375559.0,0.0,48848591.8,0.0,6432.97,10506.58,961199.96
3.458869e-08,0.0,0.0,451599.08,1552264.34,0.00,37329151.5,1.67,12375.44,141388.51,657388.46,...,1826228.34,1084.13,95.33,225788192.0,0.0,47974961.3,0.0,6432.84,10506.57,844414.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9.180324e+01,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00
9.383557e+01,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00
9.591290e+01,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00


All the power densities just turn to 0 really quickly. Why?

In [69]:
px.scatter(power_densities_df['63Ni'],  
 labels={'value':'Power Density (W/g)'})#, log_x=True, log_y=True)

In [ ]:
def plot_isotope(isotope):
    plot = go.Figure()
    plot.add_trace(go.Scatter(x = power_densities_df.index,


In [49]:
fig = go.Figure()

fig.update_layout(
    updatemenus=[
        dict(buttons=list([
            dict(label="None",
                 args=["shapes", []]),
            dict(label="All",
                 method="relayout",
                 args=["shapes", cluster0 + cluster1 + cluster2])
        ]),
        )
    ]
)

In [50]:
np.random.seed(1)

x0 = np.random.normal(2, 0.4, 400)
y0 = np.random.normal(2, 0.4, 400)
x1 = np.random.normal(3, 0.6, 600)
y1 = np.random.normal(6, 0.4, 400)
x2 = np.random.normal(4, 0.2, 200)
y2 = np.random.normal(4, 0.4, 200)

# Create figure
fig = go.Figure()

# Add traces
fig.add_trace(
    go.Scatter(
        x=x0,
        y=y0,
        mode="markers",
        marker=dict(color="DarkOrange")
    )
)

fig.add_trace(
    go.Scatter(
        x=x1,
        y=y1,
        mode="markers",
        marker=dict(color="Crimson")
    )
)

fig.add_trace(
    go.Scatter(
        x=x2,
        y=y2,
        mode="markers",
        marker=dict(color="RebeccaPurple")
    )
)

# Add buttons that add shapes
cluster0 = [dict(type="circle",
                            xref="x", yref="y",
                            x0=min(x0), y0=min(y0),
                            x1=max(x0), y1=max(y0),
                            line=dict(color="DarkOrange"))]
cluster1 = [dict(type="circle",
                            xref="x", yref="y",
                            x0=min(x1), y0=min(y1),
                            x1=max(x1), y1=max(y1),
                            line=dict(color="Crimson"))]
cluster2 = [dict(type="circle",
                            xref="x", yref="y",
                            x0=min(x2), y0=min(y2),
                            x1=max(x2), y1=max(y2),
                            line=dict(color="RebeccaPurple"))]

fig.update_layout(
    updatemenus=[
        dict(buttons=list([
            dict(label="None",
                 method="relayout",
                 args=["shapes", []]),
            dict(label="Cluster 0",
                 method="relayout",
                 args=["shapes", cluster0]),
            dict(label="Cluster 1",
                 method="relayout",
                 args=["shapes", cluster1]),
            dict(label="Cluster 2",
                 method="relayout",
                 args=["shapes", cluster2]),
            dict(label="All",
                 method="relayout",
                 args=["shapes", cluster0 + cluster1 + cluster2])
        ]),
        )
    ]
)

# Update remaining layout properties
fig.update_layout(
    title_text="Highlight Clusters",
    showlegend=False,
)
